In [ ]:
!pip install streamlit pyngrok python-docx openai
!pip install gradio openai python-docx pdfplumber fpdf

In [ ]:
import gradio as gr
import openai
from docx import Document
import pdfplumber
from fpdf import FPDF
import os
import tempfile

In [ ]:
openai.api_key = #"your_openai_api_key_here"

In [ ]:
def extract_resume_text(file):
    file_name = file.name
    if file_name.endswith(".pdf"):
        with pdfplumber.open(file.name) as pdf:
            return "\n".join([page.extract_text() or "" for page in pdf.pages])
    elif file_name.endswith(".docx"):
        doc = Document(file.name)
        return "\n".join([para.text for para in doc.paragraphs])
    else:
        return "Unsupported file format."

In [ ]:
def generate_cover_letter(file, job_description):
    try:
        resume_text = extract_resume_text(file)

        prompt = f"""
Write a professional, personalized cover letter based on the following details:

Resume:
{resume_text}

Job Description:
{job_description}

The cover letter should be well-structured, role-specific, and highlight relevant strengths.
"""

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500
        )

        cover_letter = response.choices[0].message.content

        # Save as .docx
        docx_path = "/tmp/cover_letter.docx"
        doc = Document()
        doc.add_heading("Cover Letter", level=1)
        doc.add_paragraph(cover_letter)
        doc.save(docx_path)

        # Save as .pdf
        pdf_path = "/tmp/cover_letter.pdf"
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        for line in cover_letter.split("\n"):
            pdf.multi_cell(0, 10, line)
        pdf.output(pdf_path)

        return cover_letter, docx_path, pdf_path

    except Exception as e:
        return f"❌ Error: {str(e)}", None, None


In [ ]:
# Gradio interface
interface = gr.Interface(
    fn=generate_cover_letter,
    inputs=[
        gr.File(label="Upload Resume (PDF or Word)", file_types=[".pdf", ".docx"]),
        gr.Textbox(label="Paste Job Description (optional)", lines=8, placeholder="Paste the job description here...")
    ],
    outputs=[
        gr.Textbox(label="📄 Generated Cover Letter", lines=20),
        gr.File(label="📥 Download as DOCX"),
        gr.File(label="📥 Download as PDF")
    ],
    title="AI Cover Letter Generator from Resume + Job Description",
    description="Upload your resume and optionally provide a job description to generate a personalized cover letter using OpenAI GPT."
)

In [ ]:

interface.launch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://455fdc9c2c26d30f57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
